In [3]:
from pathlib import Path
from typing import List

def print_file_content(file_path: Path):
    print(f"\n\nFile: '{file_path}'")
    print("```\n",file_path.read_text(), "\n```")

def process_files(file_patterns: List[str]):
    for pattern in file_patterns:
        for file in Path().glob(pattern):
            print_file_content(file)

file_patterns = ["../../src/*.ts", "../../package.json"]
process_files(file_patterns)



File: '../../src/streamUtils.ts'
```
 import axios, { AxiosResponse } from "axios";
import { CreateChatCompletionResponse } from "openai";
import { CancellationToken, window } from "vscode";
import { FinishReason } from "./finishReason";

export async function* streamChatCompletion(
  response: AxiosResponse<CreateChatCompletionResponse, AsyncIterable<Buffer>>,
  token: CancellationToken
): AsyncGenerator<string | FinishReason, void, undefined> {
  // types are unfortunately not well defined so we have to cast to unknown first to get an AsyncIterable<T>
  const dataStream = response.data as unknown as AsyncIterable<Buffer>;

  for await (const chunk of dataStream) {
    if (token.isCancellationRequested) {
      throw new axios.Cancel("ChatCompletion API request cancelled by user");
    }

    const lines = chunk
      .toString("utf8")
      .split("\n")
      .filter((line) => line.trim().startsWith("data: "));

    for (const line of lines) {
      const message = line.replace(/^d

Please modify the implementation of the VSCode extension above so that all OpenAI client parameters can be configured by the user. Just like temperature and model, the parameters are expected to be saved and loaded from notebook metadata, and each parameter should have its own own dedicated command.

Here's the modified implementation of the VSCode extension that allows users to configure all OpenAI client parameters. Each parameter has its own dedicated command, and the parameters are saved and loaded from the notebook metadata.

First, update the `package.json` file to add new commands for each parameter:

```json
{
  ...
  "contributes": {
    "commands": [
      ...
      {
        "command": "notebook-chatcompletion.setTopP",
        "title": "Set Top P",
        "icon": "$(list-flat)"
      },
      {
        "command": "notebook-chatcompletion.setN",
        "title": "Set N",
        "icon": "$(list-numbered)"
      },
      {
        "command": "notebook-chatcompletion.setStream",
        "title": "Set Stream",
        "icon": "$(broadcast)"
      },
      {
        "command": "notebook-chatcompletion.setMaxTokens",
        "title": "Set Max Tokens",
        "icon": "$(list-ordered)"
      },
      {
        "command": "notebook-chatcompletion.setPresencePenalty",
        "title": "Set Presence Penalty",
        "icon": "$(pulse)"
      },
      {
        "command": "notebook-chatcompletion.setFrequencyPenalty",
        "title": "Set Frequency Penalty",
        "icon": "$(pulse)"
      },
      {
        "command": "notebook-chatcompletion.setLogitBias",
        "title": "Set Logit Bias",
        "icon": "$(graph)"
      },
      {
        "command": "notebook-chatcompletion.setUser",
        "title": "Set User",
        "icon": "$(person)"
      }
    ],
    ...
  },
  ...
}
```

Next, update the `src/extension.ts` file to add new functions for each command:

```typescript
import {
  ...
} from "vscode";
import {
  ...
} from "./cellUtils";
import { CompletionType } from "./completionType";
import { FinishReason } from "./finishReason";
import { bufferWholeChunks, streamChatCompletion } from "./streamUtils";
import { UIProgress } from "./uiProgress";

export async function activate(context: ExtensionContext) {
  context.subscriptions.push(
    ...
  );

  context.subscriptions.push(
    commands.registerCommand(
      "notebook-chatcompletion.setTopP",
      setTopP
    )
  );

  context.subscriptions.push(
    commands.registerCommand(
      "notebook-chatcompletion.setN",
      setN
    )
  );

  context.subscriptions.push(
    commands.registerCommand(
      "notebook-chatcompletion.setStream",
      setStream
    )
  );

  context.subscriptions.push(
    commands.registerCommand(
      "notebook-chatcompletion.setMaxTokens",
      setMaxTokens
    )
  );

  context.subscriptions.push(
    commands.registerCommand(
      "notebook-chatcompletion.setPresencePenalty",
      setPresencePenalty
    )
  );

  context.subscriptions.push(
    commands.registerCommand(
      "notebook-chatcompletion.setFrequencyPenalty",
      setFrequencyPenalty
    )
  );

  context.subscriptions.push(
    commands.registerCommand(
      "notebook-chatcompletion.setLogitBias",
      setLogitBias
    )
  );

  context.subscriptions.push(
    commands.registerCommand(
      "notebook-chatcompletion.setUser",
      setUser
    )
  );
}

...

async function setTopP() {
  const editor = window.activeNotebookEditor!;
  const topP = await window.showInputBox({
    prompt: "Enter the Top P value (0-1):",
    validateInput: (value) =>
      parseFloat(value) >= 0 && parseFloat(value) <= 1
        ? null
        : "Top P must be between 0 and 1",
  });

  if (topP) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          top_p: parseFloat(topP),
        },
      }),
    ]);
    await workspace.applyEdit(edit);
  }
}

Continue

```typescript
async function setN() {
  const editor = window.activeNotebookEditor!;
  const n = await window.showInputBox({
    prompt: "Enter the N value (integer):",
    validateInput: (value) =>
      parseInt(value) > 0 ? null : "N must be a positive integer",
  });

  if (n) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          n: parseInt(n),
        },
      }),
    ]);
    await workspace.applyEdit(edit);
  }
}

async function setStream() {
  const editor = window.activeNotebookEditor!;
  const stream = await window.showQuickPick(["true", "false"], {
    placeHolder: "Select the Stream value:",
  });

  if (stream) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          stream: stream === "true",
        },
      }),
    ]);
    await workspace.applyEdit(edit);
  }
}

async function setMaxTokens() {
  const editor = window.activeNotebookEditor!;
  const maxTokens = await window.showInputBox({
    prompt: "Enter the Max Tokens value (integer):",
    validateInput: (value) =>
      parseInt(value) > 0 ? null : "Max Tokens must be a positive integer",
  });

  if (maxTokens) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          max_tokens: parseInt(maxTokens),
        },
      }),
    ]);
    await workspace.applyEdit(edit);
  }
}

async function setPresencePenalty() {
  const editor = window.activeNotebookEditor!;
  const presencePenalty = await window.showInputBox({
    prompt: "Enter the Presence Penalty value (0-1):",
    validateInput: (value) =>
      parseFloat(value) >= 0 && parseFloat(value) <= 1
        ? null
        : "Presence Penalty must be between 0 and 1",
  });

  if (presencePenalty) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          presence_penalty: parseFloat(presencePenalty),
        },
      }),
    ]);
    await workspace.applyEdit(edit);
  }
}

async function setFrequencyPenalty() {
  const editor = window.activeNotebookEditor!;
  const frequencyPenalty = await window.showInputBox({
    prompt: "Enter the Frequency Penalty value (0-1):",
    validateInput: (value) =>
      parseFloat(value) >= 0 && parseFloat(value) <= 1
        ? null
        : "Frequency Penalty must be between 0 and 1",
  });

  if (frequencyPenalty) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          frequency_penalty: parseFloat(frequencyPenalty),
        },
      }),
    ]);
    await workspace.applyEdit(edit);
  }
}

async function setLogitBias() {
  const editor = window.activeNotebookEditor!;
  const logitBias = await window.showInputBox({
    prompt: "Enter the Logit Bias value (JSON object):",
    validateInput: (value) => {
      try {
        JSON.parse(value);
        return null;
      } catch (error) {
        return "Logit Bias must be a valid JSON object";
      }
    },
  });

  if (logitBias) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          logit_bias: JSON.parse(logitBias),
        },
      }),
    ]);
    await workspace.applyEdit(edit);
  }
}

async function setUser() {
  const editor = window.activeNotebookEditor!;
  const user = await window.showInputBox({
    prompt: "Enter the User value (string):",
    validateInput: (value) =>
      value.trim().length > 0 ? null : "User value cannot be empty",
  });

  if (user) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          user: user,
        },
      }),
    ]);
        await workspace.applyEdit(edit);
  }
}

continue

```typescript
async function chatCompletion() {
  ...
  const topP = editor.notebook.metadata.custom?.top_p ?? 0.9;
  const n = editor.notebook.metadata.custom?.n ?? 1;
  const stream = editor.notebook.metadata.custom?.stream ?? false;
  const maxTokens = editor.notebook.metadata.custom?.max_tokens ?? 4096;
  const presencePenalty = editor.notebook.metadata.custom?.presence_penalty ?? 0;
  const frequencyPenalty = editor.notebook.metadata.custom?.frequency_penalty ?? 0;
  const logitBias = editor.notebook.metadata.custom?.logit_bias ?? {};
  const user = editor.notebook.metadata.custom?.user ?? "User";
  ...
}
```

With these changes, the VSCode extension now allows users to configure all OpenAI client parameters. Each parameter has its own dedicated command, and the parameters are saved and loaded from the notebook metadata.

Add all these new commands to the "notebook/cell/title" menu

To add all the new commands to the "notebook/cell/title" menu, update the `package.json` file by adding the new commands to the `menus` section:

```json
{
  ...
  "contributes": {
    ...
    "menus": {
      "notebook/cell/title": [
        {
          "command": "notebook-chatcompletion.chatCompletion",
          "when": "notebookType == chatcompletion",
          "group": "0_chatcompletion"
        },
        {
          "command": "notebook-chatcompletion.setModel",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setTemperature",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setTopP",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setN",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setStream",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setMaxTokens",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setPresencePenalty",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setFrequencyPenalty",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setLogitBias",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        },
        {
          "command": "notebook-chatcompletion.setUser",
          "when": "notebookType == chatcompletion",
          "group": "1_parameters"
        }
      ]
    },
    ...
  },
  ...
}
```

Now, all the new commands will be available in the "notebook/cell/title" menu.